In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

In [5]:
latent_size = 4
prior = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_size))

In [6]:
encoder = Sequential([
    Dense(64, activation='relu', input_shape=(12,)),
    Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
    tfpl.MultivariateNormalTriL(latent_size),
    tfpl.KLDivergenceAddLoss(prior)
])

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


The `KLDivergenceAddLoss` layer does not changes its input in any way. It is just a passthrough layer. This layer requires the previous layer in the model to output a distribution. It adds the KL divergence automatically to the loss function of the model. Now, we only need to take care of the reconstruction loss. 

In [7]:
decoder = Sequential([
    Dense(64, activation='relu', input_shape=(latent_size,)),
    Dense(tfpl.IndependentNormal.params_size(12)),
    tfpl.IndependentNormal(12)
])

The outptut is a IndependentNormal with dimension 12, which is the same size as the data exmaples. 

In [ ]:
vae = Model(inputs=encoder.input, outputs=decoder(encoder.output))

Now, we can define an end-to-end encoder/decoder architecture.

In [ ]:
vae.compile(loss=lambda x, pred: -pred.log_prob(x))
vae.fit(train_data, epochs=20)

Since the KL term was already added to the loss by the `KLDivergenceAddLoss` layer, we can compile the VAE model object passing in the reconstruction loss to the loss argument. To train the model we just call the fit method.

In this case the inputs and outputs that we pass as the train_data are the same thing, the data example x. The data example x is passed through the encoder, which returns the posterior approximation, we then sample from this distribution and pass the sample through the decoder which then returns the model prediction, which is the Independent Normal distribution.

In [9]:
encoder = Sequential([
    Dense(64, activation='relu', input_shape=(12,)),
    Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
    tfpl.MultivariateNormalTriL(latent_size),
    tfpl.KLDivergenceAddLoss(prior, weight=10, use_exact_kl=False)
])

The weight keyword argument can be used to include a factor to be multiplied to the KL loss term before adding it to the reconstruction loss. The default is None, where the weight is set to 1. By passing in the factor 10 we are making the KL term more important in our loss function. It is also possible to pass a tensor or array of weights, to set different weights for each element of a batch. If the `use_exact_kl` is set to false, the model will use Monte Carlo samples from the posterior. 

`approx_posterior.log_prob(approx_posterior_sample)`
 `- prior.log_prob(approx_posterior_sample)`
 
 Remember that this MC estimate is computed by evaluating the log probability of a sample from the approximate posterior according to the appropriate posterior itself and the prior. What actually happens is that the `KLDivergenceAddLoss` function extracts a tensor from the `MultivariateNormalTriL` layer according to the `convert_to_tensor_fn` assigned to that layer. The default for this argument is the distribution.sample method so this would compute the KL divergence as we would expect. If instead of the sample method, one was using the mean or mode this would be the tensor used in the approximation. So the result could be wrong.

In [ ]:
encoder = Sequential([
    Dense(64, activation='relu', input_shape=(12,)),
    Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
    tfpl.MultivariateNormalTriL(latent_size),
    tfpl.KLDivergenceAddLoss(prior, weight=10, use_exact_kl=False,
                            test_points_fn=lambda q: q.sample(10),
                            test_points_reduce_axis=0) # None, ()
])

Another way to do this is by overriding the method in the previous distribution by using the keyword argument `test_points_fn`. If this argument is provided this is the function that will be used to compute the estimate. Here, we are using a lambda function that draws 10 samples from the posterior to compute the MC KL estimate. The keyword argument `test_points_reduce_axis` specifies which axis of the KL divergence estimate tensor should be reduced out with the mean operation and which should be reduced by summing. If we set the number of samples to 10, the resulting tensor will have the number of samples in the first dimension and the batch size in the second. If we set `test_points_reduce_axis` to 0 means that this tensor will be reduced to a scalar by averaging over the sample axis and summing over the batch axis. One could also define it to None, in which all axis would be average or set it to an empty tuple, in which all axis would be summed over. One important point is that these arguments are only used for the KL divergence computation, they don't have any effect on the resulting distribution of the encoder. In this case the encoder will still draw one sample from the approximate posterior to pass to the decoder network.

In [ ]:
encoder = Sequential([
    Dense(64, activation='relu', input_shape=(12,)),
    Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
    tfpl.MultivariateNormalTriL(latent_size,
                               activity_regularizer=tfpl.KLDivergenceRegularizer(prior, 
                                                                                 weight=10, 
                                                                                 use_exact_kl=False,
                                                                                 test_points_fn=lambda q: q.sample(10),
                                                                                 test_points_reduce_axis=0)) # None, ()
])

Above you have a different way to do exactly what we previously implemented. It works by passing the same arguments to a activity regularizer which is an argument of the Multivariate Normal itself.